In [1]:
import numpy as np

In [48]:
#M = np.asarray(np.asmatrix('2 1; 1 2'))
#q = np.asarray([-5, -6], dtype=np.float64)

M = np.asarray(np.asmatrix('0.4313 0.5902; 0.5902 1.5934'))
q = np.asarray([-0.6787, -0.7577], dtype=np.float64)

In [75]:
def lcpipm(M, x, s, mu, epsilon, tau, theta, n):
    i = 0
    j = 0
        
    bm = np.zeros(shape=(n + n, n + n))
    
    bm[:n, :n] = -M
    I = np.eye(n)
    bm[:n, n:] = I
    
    #display(bm)
    
    rhs = np.zeros(shape=(n + n)).T
    
    while mu*n >= epsilon and i < 10:
        i += 1
        mu = (1 - theta)*mu;
        v = np.sqrt((x.T*s)/mu)
        p = kernel_func(v)
        while p > tau and j < 50:
            j += 1
            X = np.diag(x)
            S = np.diag(s)
            
            dv = kernel_func_dv(v)
            
            bm[n:, :n] = S
            bm[n:, n:] = X
        
            rhs[n:] = -mu*v*dv
                        
            xs = np.linalg.solve(bm, rhs)
            #display(xs)
            delta_x = xs[:n]
            #display(delta_x)
            delta_s = xs[n:]
            #display(delta_s)
            
            alpha_x, alpha_s, alpha_k = get_steplength(x, s, delta_x, delta_s, 0.85, n)
            
            x += alpha_x*delta_x
            s += alpha_s*delta_s
            
            v = np.sqrt((x.T*s)/mu)
            p = kernel_func(v)    
            
    return x, s, i*j


def get_steplength(x, s, delta_x, delta_s, eta, n):
    alpha_x = 1.0
    alpha_s = 1.0
    for i in range(n):
        if delta_x[i] < 0:
            # get alpha primal
            alpha_x = min(alpha_x, -x[i] / delta_x[i])
        if delta_s[i] < 0:
            # get alpha dual
            alpha_s = min(alpha_s, -s[i] / delta_s[i])
    # choose smallest alpha
    alpha_x = min(1.0, eta * alpha_x)
    alpha_s = min(1.0, eta * alpha_s)
    alpha_k = min(1.0, eta * min(alpha_s, alpha_x))

    return alpha_x, alpha_s, alpha_k


def kernel_func(v):
    return sum(((np.power(v, 2) - 1)/2)+ 2*np.log(1 + np.reciprocal(v)) - 2*np.log(2))

def kernel_func_dv(v):
    return sum((v - (2/(np.power(v, 2) + v))))

In [74]:
epsilon = 10e-6
tau = 2
theta = 0.5

x0 = np.asarray([0.7431, 0.3922], dtype=np.float64)
#s0 = np.dot(M, x0) + q
s0 = np.asarray([0.6555, 0.1712], dtype=np.float64)

display(s0)

n = M.shape[0]

mu0 = np.dot(x0.T, s0)/n
v0 = np.sqrt(np.multiply(x0, s0)/mu0)
display(v0)

lcpipm(M=M, x=x0, s=s0, mu=mu0, epsilon=epsilon, tau=tau, theta=theta, n=n)

array([0.6555, 0.1712])

array([1.32578598, 0.49223118])

(array([4.31468017e+00, 2.13163061e-03]), array([1.96570417, 1.65761167]), 500)

In [ ]:
test_v = np.asarray([3, 3], dtype=np.float64)
display(test_v)
display(np.power(test_v, 2))
display((np.power(test_v, 2) - 1)/2)
display(1 + np.reciprocal(test_v))
display(np.log(1 + np.reciprocal(test_v)))

kernel_func(test_v)

In [ ]:
x0 = np.asarray([3, 3], dtype=np.float64)
s0 = np.dot(M, x0) + q